# RAG with HANA Vector Store

This notebook walks you through building a **Retrieval-Augmented Generation (RAG)** application using:
- **HANA Vector Store**
- **OpenAI GPT-4o**

## Setup

### Dependencies
Ensure the libraries mentinoed in the `requirements.txt` file are installed.


### .env Variables
Create a `.env` file with the following:
```
HANA_ADDRESS=your_hana_host
HANA_PORT=your_port
HANA_USER=your_user
HANA_PASSWORD=your_password
HANA_AUTOCOMMIT=true
HANA_SSL_CERT_VALIDATE=false

AICORE_AUTH_URL=your_aicore_auth_url
AICORE_CLIENT_ID=your_aicore_client_id
AICORE_CLIENT_SECRET=your_aicore_secret
AICORE_RESOURCE_GROUP=your_resource_group
AICORE_BASE_URL=your_base_url
```

In [1]:
# Import section
import os
from dotenv import load_dotenv
load_dotenv(override=True)

from hana_ml import ConnectionContext

from gen_ai_hub.proxy.core.proxy_clients import get_proxy_client
from gen_ai_hub.proxy.native.openai import embeddings

## Retrieval

Use the user's query to retrive most semantically similar documents from the vector store to create context that will be used to ground LLM for answer generation.

In [2]:
# Connect to SAP HANA
cc = ConnectionContext(
    address=os.environ.get("HANA_ADDRESS"),
    port=os.environ.get("HANA_PORT"),
    user=os.environ.get("HANA_USER"),
    password=os.environ.get("HANA_PASSWORD"),
    encrypt=True
)

cursor = cc.connection.cursor()

print(cc.hana_version())
print(cc.get_current_schema())

4.00.000.00.1715685275 (fa/CE2024.2)
USR_336RA2ZQ5LAGTHKHCKIYB945E


In [ ]:
# Initialize AI Core proxy client
proxy_client = get_proxy_client('gen-ai-hub')

def get_embedding(query):
    """
    Create embedding vector for given text.
    """
    embeds = embeddings.create(
        model_name="text-embedding-ada-002",
        input=query
    )
    return embeds.data[0].embedding

In [4]:
def run_vector_search(query, cursor, table_name, metric="COSINE_SIMILARITY", k=4):
    """
    Performs vector search on indexed documents.
    """
    try:
        query_vector = get_embedding(query)
        if not query_vector:
            raise ValueError("Failed to generate query embedding.")

        sort_order = "DESC" if metric != "L2DISTANCE" else "ASC"
        sql_query = f"""
        SELECT TOP {k} ID, MY_TEXT, MY_METADATA
        FROM {table_name}
        ORDER BY {metric}(MY_VECTOR, TO_REAL_VECTOR('{query_vector}')) {sort_order}
        """
        cursor.execute(sql_query)
        return cursor.fetchall()
    except Exception as e:
        print(f"Error during vector search: {e}")
        return []

In [ ]:
query = "How to test for fat in foods?"

# Retrieve top 4 matching docs from vector store
context_records = run_vector_search(query, cursor, "SCIENCE_DATA_MIT6", 'COSINE_SIMILARITY', 4)
# Join the content from retrieved docs
context = ' '.join([c[1] for c in context_records])


# Augment

Augment the prompt instructions by embedding retrieved context into it.

In [9]:
prompt = f"""
Use the following context information to answer to user's query.
Here is some context: {context}

Based on the above context, answer the following query:
{query}

The answer tone has to be very proffesional in nature.

If you don't know the answer, politely say that you don't know, don't try to make up an answer.
"""

# Generation

Use an LLM from Generative AI Hub to generate response for the context augmented prompt. This allows the LLM to be grounded on the context while answering.

In [8]:
from gen_ai_hub.proxy.native.openai import chat

messages = [
    {"role": "system", "content": "You are an intelligent assistant."},
    {"role": "user", "content": prompt}
]

kwargs = dict(model_name="gpt-4o", messages=messages)

response = chat.completions.create(**kwargs)

print(response.choices[0].message.content)

To test for fats in food items, a practical method involves wrapping the food in a piece of paper and crushing it. Observing for an oily patch on the paper, especially when held against light, indicates the presence of fats. Additionally, rubbing the food item directly on the paper and allowing it to dry can also reveal fats by leaving an oily residue that remains visible after any water content has evaporated. This traditional technique provides a straightforward assessment of fat content in various foods.
